In [ ]:
from google.colab import drive; drive.mount("/content/drive")

Mounted at /content/drive


In [ ]:
#pip install --upgrade tensorflow-gpu
#!pip install Keras==2.2.4
#hasil_preprocessing_kamusKita_kamusInternet_Revisi.xlsx
!pip install tokenizers

In [ ]:
import pandas as pd
import numpy as np
import math
import re
import tensorflow as tf
from sklearn.model_selection import train_test_split
#import sentencepiece as spm
from tensorflow.keras.layers import Dense, Input
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import ModelCheckpoint
#from tokenizers import ByteLevelBPETokenizer
from keras import backend as K
print("Tf",tf.__version__)

Tf 2.3.0


In [ ]:
pip install transformers

In [ ]:
from transformers import *

In [ ]:
  from transformers import XLMRobertaTokenizer
  from transformers import XLMRobertaModel

In [ ]:
MAX_LEN = 128  
AUTO = tf.data.experimental.AUTOTUNE
PATH = "/content/drive/My Drive/Gemastik/XLMRoberta/XLMROBERTA.h5"

tokenizer = XLMRobertaTokenizer.from_pretrained("jplu/tf-xlm-roberta-base",lowercase = True)

In [ ]:

#traindf = pd.read_excel("/content/drive/My Drive/hasil_preprocessing_kamusKita_kamusInternet (1).xlsx")
traindf = pd.read_excel("/content/drive/My Drive/hasil_preprocessing_kamusKita_kamusInternet_Revisi (1).xlsx")
traindf.drop(["Unnamed: 0"],axis = 1,inplace = True)
 #traindf = pd.read_csv("/content/drive/My Drive/Gemastik/DatasetLabeled.csv",delimiter = "\t")
 #traindf.head()
 #traindf5000 = traindf[:5000]

In [ ]:
traindf["sentiment"].unique()

array([1, 0])

In [ ]:
print(traindf.columns)
traindf = traindf[["list_text_after_stop_word","sentiment"]]
traindf.dropna(inplace = True)
traindf["sentiment"] = traindf["sentiment"].astype('float32')
print(len(traindf))
traindf.head()

Index(['text', 'sentiment', 'list_text_after_stop_word',
       'hasil_akhir_preprocessing'],
      dtype='object')
3551


,list_text_after_stop_word,sentiment
0,stayathome withmestudy pembahasan ujian sekola...,1.0
1,nasehat pemuda daily voice letter dikirimkan g...,1.0
2,kampusny elearning brother kaget belajar darin...,1.0
3,hmm kemarin teman sekelas telepon iya intinya ...,0.0
4,serius stres banget ngadepin pembelajaran jara...,0.0


In [ ]:

dick = {"kampusny" : "kampusnya",
        "brother" : "saudara",
        "berbondong2" : "berbondong-berbondong",
        "pup" : "buang air besar",
        "bb" : "berat badan",
        "rebahan" : "tidur",
        "wkwk" : "",
        "atuser" : "",
        "joki" : "calo",
        "fismat" : "",
        "jajaki" : "",
        "berbulan2" : "berbulan-bulan",
        "sekolah2" : "sekolah-sekolah",
        "bm" : "gelisah",
        "gupuh" : "panik",
        "seenggaknya" : "setidaknya",
        "bener²" : "benar-benar",
        "bener2" :"benar-benar",
        "nurut-nurut" : "nurut",
        "naudzubillah minzalik" : "astaga",
        "investeam" : "",
        "mantul" : "mantap",
        "bimtek" : "bimbingan",
        "oflen" : "luring",
        "iyakaan" : "iya kan",
        "na" : "nya","hiks" : "sedih",
        "kegoda" : 'tergoda',"gais" : "ayo",
        "gseru" : "tidak seru","grup" : "kelompok","grub" :"kelompok",
        "grup" : "kelompok",
        "wassap" : "whatsapp","wa" : "whatsapp","meng" :"jangan",
        "bethol" :'betul',"pusyink" : "pusing","onlinegabutt" : "daring malas",
        "berbedabeda" :"berbeda beda" , "guoblokk " : "bodoh" , "anjir" : "anjing",
        "tidooooor" : "tidur" , "deg2ann" : "gugup" , "tggl" : "tanggal",
        "uwuu" : "wow" , "uwu" : "wow" , "sekolahkuliah" : "sekolah kuliah"
        ,"semangaaat " : "semangat" , "aloo" : "hai" , "njelasin" : "jelaskan",
        "ku" : "saya","kayanyaaa" : "sepertinya","pagi2" : "pagi-pagi" , "siap2" : "siap-siap",
        "ngamaooo" : "enggak mau", "b4it8km00022020" :"","pengin":"ingin","ig":"instagram",
        "berbulanbulan" : "bulan-bulan" ,  "niiih" : "nih","anakanaknya" : "anak-anak",
        "belajardaring" : "belajar daring","boro²" : "boro-boro","studi" : "belajar","budoh" : "bodoh",
        "tibatiba" : "tiba-tiba","crush" : "sayang","maju2" : "maju-maju","amjink" : "anjing",
        "bobrok" : "susah" , "🤩" : "" , "gform" : "formulir"  , "tidurrr" : "tidur",
        "edan" : "luar biasa" , "ukk" : "ujian kenaikan kelas" , "padhal" : "padahal" , "asu" : "anjing",
        "astaghfirullah" : "aduh","negerikuambyar" : "negeri saya bercerai-berai","plisss" : "mohon","valid" :"betul",
        "mrasa" : "merasa" , "gkk" : "enggak" , "smnjk"  : "semenjak" , "jlk" : "jelek","sambat" :"ngeluh",
        "sendiri2" : "sendiri-sendiri","muak" : "benci","🤭" : "","asww" : "anjing" , "babu" : "pembantu",
        "guys" : "teman" , "apalaagi" : "apa lagi", "full" : "penuh" , "time" : "waktu", "diem²" : "diam",
        "bae" : "aja" , "gasih" : "enggak sih" ,"jlas" : "jelas","gajelas" : "tidak jelas","pdf" : "dokumen",
        "gadak" : "tidak ada" , "emosik" : "emosi","garagara" : "gara-gara" ,"plis" : "mohon", "utekku" : "otak saya",
        "gasinkron" : "sulit memahami" , "ngegim" : "olahraga","anjrd" : "anjing" , "ngapa" : "kenapa","mashook":"masuk",
        "ngadi²" : "kadang-kadang" , "balek" : "kembali","anjiiiing" : "anjing" , "mrnangis" : "menangis","hanjeng" : "anjing",
        "astagfir" : "aduh","soalsoal" : "soal-soal","onlengrgegegttttt" : "daring","no" :"tidak",
        "offlen" : "laring","anjir" : "anjing","oflen" : "laring","🥺" : "sedih","anzeng" : "anjing",
        "fucek" : "gila","uts" :"ujian", "uas" : "ujian","stupid²" : "bodoh" , "stupid" : "bodoh",
        "ra" : "tidak" ,"offline" : "laring","tijel" : "tidak jelas" , "meha" : "iya","diam2" : "diam-diam",
        "classes" : "kelas" , "jengkel" : "tidak suka" , "geng" : "kumpulan orang","semenamena" : "se-enaknya",
        "morniggg" : "selamat pagi" , "onlinea" : "daring","anjritt" : "anjing","cintaaaaaaaaaaaaah" : "cinta",
        "deg2an" : "gugup","ngecheat" : "berbuat curang","enek" : 'tidak suka',"ancur2an" : "hancur",
        "super2" : "sangat","dudul" : "bodoh","tulalit" : "bodoh","uring2an" : "pusing","bgtttttt" : "sangat",
        "gr2" : 'gara-gara',"dikit2" : "beberapa saat","rebel" :"malas" , "mingdep" : 'minggu depan',
        "tahun2" : "tahun-tahun" , "susah3" : "sulit-sulit","mlzzz" : "malas" , "gakuatt" : "enggak kuat" ,
        "pagi²" : "pagi-pagi","emak" : "ibu","mamak" : "ibu","puseng" : "pusing","handphone" : "gawai","hp" : "gawai",
        "brrti" : "berarti","ayu" : "ayo","berbulan²" : "bulan-bulan","siap2" : "siap-siap","nanti²" : "nanti","gdpt":"enggak dapat","sikit" : "sakit",
        "gamasuk" : 'enggak masuk',"syemangats" : "semangat","anjr" : "anjing","sengsara2" : 'sengsara',"suntuk" : "malas","mlsin" : "malas",
        "cringe" : "jelek","kontol" : "kelamin","bgun" : "bangun","afk" : "malas","tahi" : "kotoran","pening" : "pusing","batttt" : "banget",
        "tos" : "sudah","terooooos" : "terus","anjeng" : "anjing"

        }

deleteindex = [
          0,1,5,6,7,11,12,13,16,17,18,19,21,23,24,25,26,27,34,30,29,39,40
          ,42,43,46,51,55,59,60,61,63,67,69,70,71,73,75,79,80,81,83,84,87,91,94,100,2303,
          2305,2306,2307,2310 ,2313 ,2315 , 2317 , 2318 ,2319 ,2333,2334 ,2336,2338 , 2341 ,2342 ,2344 ,2347 ,2348,2351,2352,2354,2356,2357,2359 ,2360,2362 ,2363,2364,2365,
          2367,2368,2370,2371,2373,2375 ,2379,2382 ,2383,2386,2387,2388,2389,2392,2393 ,2394,2395,2398,
          2399,2401,2403,2405 ,2406,2407,2408,2411 ,2413,2414,
          2415,2416,2419,2420,2422,2424,2428 ,2430,2431,2434,2451,2510,2723,2758,3022,3031,
          7,2848,2849,2850,2851,2854,2855,2857,2858,2860,2861,2862,2866,2868,2871,2876,2877,2878,2883,2884,2885,2886,2887,2889,2891,2892,2893,2896,2897,2901,2902,2903,2907,2909,2910,2911,2913,2916,2917,2918,2921,2926,2927,2928,2931,2932,2933,2934,2935,2937,2939,2941,2942,2943,29414,2945,2946,2947,2948,2949,2950,2955,2957,2958,2959,2965,2966,2967,2969,2971,2972,2974,2975,2976,2977,2980,2983,2984,2987,2988,2989,2990,2991,2992,2995,2996,2998,2999,3091,3093,3094,3095,2744,2745,2746,2752,3098,3108,3115,3119,3120,3121,3125,3127,3130,3131,3133,3136,3137,3143,3144,3145,3149,3150,
          3151,3153,3154,3161,3162,3176,3179,3185,3191,3190,3192,2548,2621,2639,2717,3013,
          3170,3195,3196,3200,3202,3205,3206,3207,3208,3209,3216,2452,2481,2499 ,2523,2536,2644 ,2783,2784,2874,
          3222,3223,3224,3225,3228,3229,3230,3232,3237,3238,3230,3232,3235,3237,3238,3239,3242,3244,3246,3247,3249,3250,
          3241,3251,3252,3256,3266,3267,3268,3272,2432,2437,2448,2447,2449,2461,2492,2511,2544,2564,2568,2569,2611,3061,3074,3079,
          3273,3276,3279,3282,3284,3285,3288,3293,3294,3295,3296,3297,3300,3303,3306,3309,3310,3319,3324,3325,3326,3327,3328,2792,
          3331,3332,3334,3336,3338,3339,3340,3342,3346,3350,3352,2302,2418,2473,2474,2479,2505,2520,2525,2534,2549,2698,2775,3337,
          3354,3355,3356,3357,3359,3362,3365,3368,3369,3374,3375,3378,3380,3385,3389,3394,3395,3401,3403,3407,3408,3409,3418,3419,3425,3431,3433,3434,3440,3442,3444,3448,3451,3453,3458,3459,3458,3459,3460,3464,3471,3483 ,3488,3489,3491,3495,3499,3500,3502,3503,3506,3507,3512,3513,3515,3518,3520,3522,3523,3524,3525,3530,3531,3535,3536,3541,3543,
        ]

def appliedSingkatan(text):
    texts = text.split()
    wordFin = ""
    for word in texts:
        if word in dick.keys():
            theword = dick[word]
            wordFin = wordFin + " " + theword
        if word not in dick.keys():
            wordFin = wordFin + " " + word
    return wordFin

text = "anjir ku teh"
appliedSingkatan(text)

#noted = [68,74,86,88,98,99,2343 ,2418,

' anjing saya teh'

In [ ]:
traindfCutted = traindf.loc[2301:3551,:]
traindfCutted["list_text_after_stop_word"] = traindfCutted["list_text_after_stop_word"].apply(appliedSingkatan)

traindfCutted.loc[traindfCutted.index.isin(deleteindex),"sentiment"] = np.nan
traindfCutted.dropna(inplace=True)
len(traindfCutted)


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:966: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#retu

873

In [ ]:
for i in (traindfCutted.index):
  print(f"{i} --> {traindfCutted.loc[i,'list_text_after_stop_word']} , {traindfCutted.loc[i,'sentiment']} ")

2301 -->  susah pembelajaran jarak jauh ujian betul , 1.0 
2304 -->  sekolah daring malas , 0.0 
2308 -->  belajar salah satunya sih belajar nya daring , 1.0 
2309 -->  belajar ujian daring pusing tidur awkwkwk , 0.0 
2311 -->  sekolah daring malas , 0.0 
2312 -->  daerah dipaksakan menerapkan pembelajaran jarak jauh kondisinya berbeda beda , 1.0 
2314 -->  sekolah daring guoblokk anjirr , 0.0 
2316 -->  kelulusan lengkap perayaan ujian langsung sekolah langsung wisuda daring , 0.0 
2320 -->  sekolah daring paham paham materinya , 1.0 
2321 -->  pembelajaran daring dikembangkan luas , 0.0 
2322 -->  profesional kontra proses pembelajaran daring marak sisi belajar sisi belajar semudah , 1.0 
2323 -->  dear corona jarang kuota hospot muncul ayolah bersahabat sih lambat belajar disekolah bingung kbm daring , 0.0 
2324 -->  disaat kuliah klimax praktikum terancam daring , 1.0 
2325 -->  cegah penyebaran virus corona unsyiah laksanakan pembelajaran daring , 0.0 
2326 -->  ya iya ayo tidur b

In [ ]:
wrongNeg = [2321,3426,3411,2358,2562,2476,2914,2609,2480,3466,2541,2875,3011,2584,2528,2576,2870,2482,3415,2650,2486,2501,2680,2513,2537,2749,3386,3139,3101,3203,2730,2560,3366,3290,3257,3123]
WronglPos = [2301,3117,3062,2539,2540,2312,2512,2327,2631,2329,2339,2374,2376,2435,2460]

for i in wrongNeg:
  traindfCutted.loc[i,"sentiment"] = 1.0
for i in WronglPos:
  traindfCutted.loc[i, "sentiment"] = 0.0

traindfCutted.head()

/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:966: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


,list_text_after_stop_word,sentiment
2301,susah pembelajaran jarak jauh ujian betul,0.0
2304,sekolah daring malas,0.0
2308,belajar salah satunya sih belajar nya daring,1.0
2309,belajar ujian daring pusing tidur awkwkwk,0.0
2311,sekolah daring malas,0.0


In [ ]:
input_index = [1197,1188,1182,1179,1170,1166,1164,1149,1146,1125,1122,1115,1112,1108,1106,1104,1084,1075,1062,1054,1051,1045,1043,1042,1038,
               1032,1016,1012,1005,1003,987,981,977,976,971,969,967,961,957,953,947,941,940,938,937,932,929,921,920,918,910,906,904,900,896,
               883,878,874,868,865,859,858,854,836,825,822,807,806,805,802,799,797,792,791,788,776,774,772,768,762,761,760,759,758,756,755,
               753,749,746,745,742,736,733,728,722,721,720,710,708,707,704,695,694,685,680,675,671,669,668,661,660,659,655,654,653,649,646,
               644,643,637,636,635,628,625,623,622,621,603,598,594,591,590,589,584,583,577,576,572,570,568,566,565,563,562,561,560,555,556,
               554,552,551,550,545,543,542,541,539,538,536,535,534,533,531,530,523,519,517,516,510,507,508,505,504,502,501,499,497,494,493,
               488,487,486,485,480,479,477,476,475,470,467,463,462,460,458,456,454,452,447,445,443,441,440,437,435,431,430,429,427,426,421,
               414,413,408,407,406,404,402,401,396,388,381,380,379,378,376,373,372,370,368,365,362,360,355,354,351,348,343,342,340,339,338,
               337,336,335,333,328,323,321,316,315,314,312,309,308,306,305,304,302,301,298,283,282,281,278,279,277,274,270,267,266,257,254,
               255,252,250,248,247,246,243,242,241,233,232,231,228,218,217,209,207,205,202,201,200,198,194,192,191,189,188,187,186,181,178,173,171,172,170,
               168,167,163,157,155,150,146,146,143,141,138,136,135,132,131,125,120,118,117,116,115,114,112,111,110,109,108,102,103,104,107]

fendDataset = traindf.loc[traindf.index.isin(input_index),:]
fendDataset.head()

FixedData = pd.concat([traindfCutted, fendDataset],ignore_index=False)
np.sum(FixedData["sentiment"] == 0) , np.sum(FixedData["sentiment"] == 1)
pd.set_option("max_rows",None)
FixedData
FixedData.to_excel("/content/drive/My Drive/fixedData.xlsx")

In [ ]:
def regular_encode(texts, tokenizer = tokenizer, maxlen=128):
    enc_di = tokenizer.batch_encode_plus(
        texts, 
        max_length=maxlen,
        return_attention_mask = True,
        return_token_type_ids = True,
        pad_to_max_length = True,
        truncation = True
        
    )
    
    return np.array(enc_di["input_ids"])
#delThis = [49 ,122,125,245,251,230,260,275,276,279,284,302,306,307,329,357,370,385,391,463,472,482,490,521,530,543,557,558,574,590,626,628,657,703,714,737,751,772,1364,1391,1392,1513,1587,1609,1614,1627,1757,1896,1904,1932,1926,1917,1952,1967,2016,2037,2047,2059,2114,2142,2146,2160,2171,2176,2200,2221,2238,2248,2255,2262,2288,2347,2366,2393,2429,2442,2451,2511,2453,2565,1808,1821,1855,1856,1888,1653,1678,1694,1695,1745,1409,1453,696,269]

#Lierdf = traindf.drop(delThis)
#Lierdf.shape

In [ ]:
x_train,x_val,y_train,y_val = train_test_split(FixedData["list_text_after_stop_word"],FixedData["sentiment"],
                                               test_size = 0.15, random_state = 0)
x_Encoded = regular_encode(x_train,tokenizer,MAX_LEN)
x_valEncoded = regular_encode(x_val,tokenizer,MAX_LEN)

/usr/local/lib/python3.6/dist-packages/transformers/tokenization_utils_base.py:1770: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


In [ ]:
from tensorflow.keras.utils import to_categorical#
y_train = to_categorical(y_train,num_classes = 2)
y_val = to_categorical(y_val,num_classes = 2)
y_train

array([[0., 1.],
       [0., 1.],
       [0., 1.],
       ...,
       [1., 0.],
       [1., 0.],
       [1., 0.]], dtype=float32)

In [ ]:
x_Encoded

array([[     0,  49985,  25396, ...,      1,      1,      1],
       [     0,   8853,  20399, ...,      1,      1,      1],
       [     0,  75236,     48, ...,      1,      1,      1],
       ...,
       [     0, 112786,   7547, ...,      1,      1,      1],
       [     0,  75236,  93937, ...,      1,      1,      1],
       [     0,  32583,  10601, ...,      1,      1,      1]])

In [ ]:
train_dataset = (
    tf.data.Dataset
    .from_tensor_slices((x_Encoded, y_train))
    .batch(32)
    .repeat()
    .prefetch(AUTO)
)

In [ ]:
valid_dataset = (
    tf.data.Dataset
    .from_tensor_slices((x_valEncoded, y_val))
    .batch(32)
    .cache()
    .prefetch(AUTO)
)

In [ ]:
from transformers import TFAutoModel

In [ ]:
import keras
def main_model():
  config = XLMRobertaConfig.from_pretrained("jplu/tf-xlm-roberta-large")
  #config.num_labels = 2
  METRICS = [
      keras.metrics.TruePositives(name='tp'),
      keras.metrics.FalsePositives(name='fp'),
      keras.metrics.TrueNegatives(name='tn'),
      keras.metrics.FalseNegatives(name='fn'), 
      keras.metrics.BinaryAccuracy(name='accuracy'),
      keras.metrics.Precision(name='precision'),
      keras.metrics.Recall(name='recall'),
      keras.metrics.AUC(name='auc'),
]  
  strategy = tf.distribute.get_strategy()

  with strategy.scope():
    ENCODER = TFAutoModel.from_pretrained("jplu/tf-xlm-roberta-large",config= config)
    input_ids = tf.keras.layers.Input(shape=(MAX_LEN), dtype=tf.int32)

    embedding, sequence_output = ENCODER(input_ids)
      #Disini Boleh Di adjust  
    logits = embedding[:,0,:]
    logits = tf.keras.layers.Dropout(0.8)(logits)
    logits = tf.keras.layers.Dense(512,tf.keras.layers.LeakyReLU())(logits)
    logits = tf.keras.layers.Dropout(0.4)(logits)
    logits = tf.keras.layers.Dense(16,tf.keras.layers.ReLU())(logits)
    logits = tf.keras.layers.Dropout(0.3)(logits)
    outputs = tf.keras.layers.Dense(units=2, activation='softmax')(logits)
    model = tf.keras.Model(inputs=input_ids, outputs=outputs)
    #optimizer = tf.keras.optimizers.SGD(lr=0.001,momentum=0.9)
    optimizer = tf.keras.optimizers.Adam(lr=0.01)
    loss = tf.keras.losses.CategoricalCrossentropy()

    for layer in model.layers[:2]:
      layer.trainable = False

    model.compile(optimizer=optimizer, loss=loss, metrics=METRICS)
  return model

model = main_model()
model.summary()

Some weights of the model checkpoint at jplu/tf-xlm-roberta-large were not used when initializing TFXLMRobertaModel: ['lm_head']
- This IS expected if you are initializing TFXLMRobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing TFXLMRobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFXLMRobertaModel were initialized from the model checkpoint at jplu/tf-xlm-roberta-large.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFXLMRobertaModel for predictions without further training.


Model: "functional_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 128)]             0         
_________________________________________________________________
tfxlm_roberta_model (TFXLMRo ((None, 128, 1024), (None 559890432 
_________________________________________________________________
tf_op_layer_strided_slice (T [(None, 1024)]            0         
_________________________________________________________________
dropout_74 (Dropout)         (None, 1024)              0         
_________________________________________________________________
dense (Dense)                (None, 512)               524800    
_________________________________________________________________
dropout_75 (Dropout)         (None, 512)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 16)               

In [ ]:
from sklearn.utils.class_weight import compute_class_weight
keys = np.unique(FixedData["sentiment"].values)
values = compute_class_weight("balanced", keys, FixedData["sentiment"].values)
cw = dict(zip(keys, values))
cw

{0.0: 0.6470588235294118, 1.0: 2.2}

In [ ]:
def create_learning_rate_scheduler(max_learn_rate=1e-5,
                                   end_learn_rate=1e-7,
                                   warmup_epoch_count=10,
                                   total_epoch_count=90):

    def lr_scheduler(epoch):
        if epoch < warmup_epoch_count:
            res = (max_learn_rate/warmup_epoch_count) * (epoch + 1)
        else:
            res = max_learn_rate*math.exp(math.log(end_learn_rate/max_learn_rate)*(epoch-warmup_epoch_count+1)/(total_epoch_count-warmup_epoch_count+1))
        return float(res)
    learning_rate_scheduler = tf.keras.callbacks.LearningRateScheduler(lr_scheduler, verbose=1)

    return learning_rate_scheduler

In [ ]:

total_epoch_count = 10
n_steps = x_train.shape[0]
history = model.fit(train_dataset,
          validation_data=valid_dataset,
          steps_per_epoch =25,
          epochs = 10,class_weight = cw,)

Epoch 1/10
25/25 [==============================] - 24s 980ms/step - loss: 0.7384 - tp: 623.0000 - fp: 177.0000 - tn: 623.0000 - fn: 177.0000 - accuracy: 0.7788 - precision: 0.7788 - recall: 0.7788 - auc: 0.7707 - val_loss: 0.6098 - val_tp: 133.0000 - val_fp: 49.0000 - val_tn: 133.0000 - val_fn: 49.0000 - val_accuracy: 0.7308 - val_precision: 0.7308 - val_recall: 0.7308 - val_auc: 0.7333
Epoch 2/10
25/25 [==============================] - 24s 968ms/step - loss: 0.7114 - tp: 586.0000 - fp: 186.0000 - tn: 586.0000 - fn: 186.0000 - accuracy: 0.7591 - precision: 0.7591 - recall: 0.7591 - auc: 0.7654 - val_loss: 0.6404 - val_tp: 133.0000 - val_fp: 49.0000 - val_tn: 133.0000 - val_fn: 49.0000 - val_accuracy: 0.7308 - val_precision: 0.7308 - val_recall: 0.7308 - val_auc: 0.7308
Epoch 3/10
25/25 [==============================] - 24s 979ms/step - loss: 0.6821 - tp: 605.0000 - fp: 167.0000 - tn: 605.0000 - fn: 167.0000 - accuracy: 0.7837 - precision: 0.7837 - recall: 0.7837 - auc: 0.7886 - val_

In [ ]:

#print(model.evaluate(x_valEncoded,y_val))

y_pred = model.predict(x_valEncoded)
result = model.evaluate(x_Encoded,y_train)
resultVal = model.evaluate(x_valEncoded,y_val)


6/6 [==============================] - 4s 610ms/step - loss: 0.6770 - tp: 133.0000 - fp: 49.0000 - tn: 133.0000 - fn: 49.0000 - accuracy: 0.7308 - precision: 0.7308 - recall: 0.7308 - auc: 0.7308


In [ ]:
model.save_weights(PATH, overwrite=True) 